In [2]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


In [3]:

app = dash.Dash(__name__)

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Read the automobiles data into pandas dataframe
# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv',
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str,
                                   'Div2Airport': str, 'Div2TailNum': str})

year_list = [i for i in range(2005, 2021, 1)]

"""Compute graph data for creating yearly airline performance report

Function that takes airline data as input and create 5 dataframes based on the grouping condition to be used for plottling charts and grphs.

Argument:

    df: Filtered dataframe

Returns:
   Dataframes to create graph.
"""
def compute_data_choice_1(df):
    # Cancellation Category Count
    bar_data = df.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
    # Average flight time by reporting airline
    line_data = df.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
    # Diverted Airport Landings
    div_data = df[df['DivAirportLandings'] != 0.0]
    # Source state count
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    # Destination state count
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
    return bar_data, line_data, div_data, map_data, tree_data


In [ ]:

#Layout Section of Dash
app.layout = html.Div(children=[
                                html.H1('Car Automobile Components',
                                         style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 24}),

                                #outer division starts
                                html.Div([
                                    # First inner divsion for  adding dropdown helper text for Selected Drive wheels
                                    html.Div(
                                        [
                                            #TASK 3B
                                            html.H2('Drive Wheels Type:', style={'margin-right': '2em'}),
                                        ]
                                    ),


                                    #TASK 3C
                                    dcc.Dropdown(
                                        id='demo-dropdown',
                                        options=[
                                            {'label': 'Rear Wheel Drive', 'value': 'rwd'},
                                            {'label': 'Front Wheel Drive', 'value': 'fwd'},
                                            {'label': 'Four Wheel Drive', 'value': '4wd'}
                                        ],
                                        value='rwd'
                                    ),

                                    #Second Inner division for adding 2 inner divisions for 2 output graphs
                                    html.Div([

                                        #TASK 3D

                                    ], style={'display': 'flex'}),


                                ]),
                                #outer division ends
                                html.Div([
                                    html.Div([ ], id='plot1'),
                                    html.Div([ ], id='plot2')
                                ], style={"display":"flex"}),

                                ])
#layout ends

#Place to add @app.callback Decorator
#TASK 3E
@app.callback([Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children')],
              Input(component_id='demo-dropdown', component_property='value'))

#Place to define the callback function .
#TASK 3F
def display_selected_drive_charts(value):



    filtered_df = auto_data[auto_data['drive-wheels']==value].groupby(['drive-wheels','body-style'],as_index=False). \
        mean()

    filtered_df = filtered_df

    fig1 = px.pie(filtered_df, values='price', names='body-style', title="Pie Chart")
    fig2 = px.bar(filtered_df, x='body-style', y='price', title='Bar Chart')

    return [dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2) ]




if __name__ == '__main__':
    app.run_server()